In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from scipy.spatial.distance import euclidean

def read_coordinates(file_path):
    """
    Reads coordinates from a .tsp file and returns a list of (latitude, longitude) tuples.
    
    Assumes each line in the file is formatted as: Node_Number Latitude Longitude
    """
    coordinates = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 3:  # Check if line has exactly three parts
                _, latitude, longitude = parts
                coordinates.append((float(latitude), float(longitude)))
            elif line.strip() == "EOF":  # Stop at the end of file marker if present
                break
    return coordinates

def calculate_distance_matrix(coordinates):
    """
    Calculates the distance matrix for a list of coordinates (latitude, longitude).
    """
    num_locations = len(coordinates)
    distance_matrix = np.zeros((num_locations, num_locations))
    
    for i in range(num_locations):
        for j in range(i + 1, num_locations):
            distance = euclidean(coordinates[i], coordinates[j])
            distance_matrix[i, j] = distance
            distance_matrix[j, i] = distance  # Symmetric matrix

    return distance_matrix

def create_data_model(distance_matrix):
    """Stores the distance matrix in the data model."""
    data = {
        "distance_matrix": distance_matrix,
        "num_cities": len(distance_matrix)
    }
    return data

def solve_tsp(distance_matrix, mip_gap=0.08):
    """
    Solve the TSP problem with Gurobi, using the provided distance matrix, a time limit,
    and an optimality gap.
    
    Args:
        distance_matrix: The distance matrix.
        time_limit (int): Time limit in seconds for the optimization run.
        mip_gap (float): The desired optimality gap (e.g., 0.1 for 10%).
    """
    # Load data
    data = create_data_model(distance_matrix)
    n = data["num_cities"]
    distance_matrix = data["distance_matrix"]

    # Create a new model
    model = gp.Model("TSP")

    # Set the time limit and MIP gap
    # model.Params.TimeLimit = time_limit
    model.Params.MIPGap = mip_gap

    # Create variables
    x = model.addVars(n, n, vtype=GRB.BINARY, name="x")
    u = model.addVars(n, vtype=GRB.CONTINUOUS, name="u")

    # Set objective
    model.setObjective(
        gp.quicksum(distance_matrix[i][j] * x[i, j] for i in range(n) for j in range(n)), GRB.MINIMIZE
    )

    # Constraints
    for i in range(n):
        model.addConstr(gp.quicksum(x[i, j] for j in range(n) if j != i) == 1, f"depart_{i}")
        model.addConstr(gp.quicksum(x[j, i] for j in range(n) if j != i) == 1, f"arrive_{i}")

    # Subtour elimination constraints (MTZ formulation)
    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                model.addConstr(u[i] - u[j] + n * x[i, j] <= n - 1, f"subtour_{i}_{j}")

    # Optimize model
    model.optimize()

    # Print the solution
    if model.status == GRB.OPTIMAL or model.status == GRB.TIME_LIMIT:
        print(f"Best route distance found: {model.objVal}")
        print("Route:")
        current_city = 0
        route = [current_city]
        for _ in range(n - 1):
            for j in range(n):
                if x[current_city, j].x > 0.5:
                    route.append(j)
                    current_city = j
                    break
        print(" -> ".join(map(str, route)) + f" -> {route[0]}")  # return to start
    else:
        print("No feasible solution found within the time limit or optimality gap.")

if __name__ == "__main__":
    # Replace 'path/to/file.tsp' with your actual .tsp file path
    coordinates = read_coordinates('Add_File_Path')
    distance_matrix = calculate_distance_matrix(coordinates)
    
    # Set the desired time limit in seconds (e.g., 120 seconds)
    #time_limit_seconds = 120
    optimality_gap = 0.08  # 10%
    solve_tsp(distance_matrix, mip_gap=optimality_gap)
